## Project Name: Orcasound Salmon
### Program Name: srkw_satellite_tagging.ipynb
### Purpose: To explore and visualize the SRKW sattelite tagging data 
##### Date Created: Mar 5th 2023

In [35]:
import os 
from os.path import join as pjoin

import datetime
import pytz
from pytz import timezone

import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None #suppress chained assignment 

import plotly.graph_objs as go

Define path and data file

In [36]:
base_path=os.path.abspath('')
data_path=pjoin(base_path, "data/srkw_satellite_tagging")
data_fname="SRKW occurrence coastal - SRKW occurrence coastal Data.csv"

Read in data

In [37]:
raw_data=pd.read_csv(pjoin(data_path, data_fname))
raw_data=raw_data.rename(columns={'Lat P':'lat_p', 'Lon P':'lon_p', 'Lat A':'lat_a','Lon A':'lon_a'})
raw_data.columns

Index(['Animal', 'Spp', 'Line', 'Thetime', 'Apfreq', 'lat_p', 'lat_a', 'lon_p',
       'lon_a', 'Lc94', 'Sen1', 'Sen2', 'Sen3', 'Sen4', 'Ptt', 'Iqx', 'Iqy',
       'Nmess', 'Gt120', 'Apbest', 'Dur', 'Nopc', 'Altitude', 'Colorname',
       'Sex', 'Age', 'Popid', 'Month', 'Day', 'Year', 'Hour', 'Minute',
       'Second', 'Date', 'Time', 'Gmt Date', 'Gmt Hour', 'Gmt Min', 'Gmt Sec',
       'Jd', 'Jday', 'Ddmm', 'Mmdd', 'Deplylat', 'Deplylon', 'Deltahr', 'Duty',
       'Dutylocs', 'Daysdply', 'Deployed', 'Record Number', 'Accessed Date',
       'Source Url', 'Preferred Citation'],
      dtype='object')

In [38]:
data=raw_data[['Animal', 'lat_p', 'lon_p', 'lat_a',
       'lon_a', 'Dur', 'Sex', 'Popid', 
       'Month', 'Day', 'Year', 'Hour', 'Minute', 'Second']]

List the individuals included in the data

In [39]:
set(data.Popid)

{'J26', 'J27', 'K 25', 'K33', 'L84', 'L87', 'L88', 'L95'}

Remove white space in 'K 25'

In [40]:
data['Popid']=data['Popid'].apply(lambda x: x.replace(' ',''))

Convert GMT time to Pacific time

In [41]:
def gmt2pst(yr,mon,day,h,m,s):
    tz=pytz.timezone('GMT')
    _date1=datetime.datetime(yr, mon, day,h,m,s, tzinfo=tz)
    _date2=_date1.astimezone(timezone('US/Pacific'))
    return(_date2)

In [42]:
data['datetime_pst']=data.apply(lambda x: gmt2pst(x.Year, x.Month, x.Day, x.Hour, x.Minute, x.Second), axis=1)
data=data.drop(columns=['Year','Month','Day','Hour','Minute','Second'])
data['date']=data.apply(lambda x: x.datetime_pst.date(), axis=1)
data['time']=data.apply(lambda x: x.datetime_pst.time(), axis=1)
data['year']=data.apply(lambda x: x.date.year, axis=1)
data['month']=data.apply(lambda x: x.date.month, axis=1)
data['day']=data.apply(lambda x: x.date.day, axis=1)
data['hour']=data.apply(lambda x: x.time.hour, axis=1)
data['minute']=data.apply(lambda x: x.time.minute, axis=1)
data['second']=data.apply(lambda x: x.time.second, axis=1)

In [43]:
data['day_of_year']=data['date'].apply(lambda x: pd.Period(x, freq='D').day_of_year)

Split data into different srkw

In [44]:
data_j26=data[data.Popid=='J26']
print('J26 Data range:', min(data_j26.datetime_pst), " ~ ",max(data_j26.datetime_pst))
data_j27=data[data.Popid=='J27']
print('J27 Data range:', min(data_j27.datetime_pst), " ~ ",max(data_j27.datetime_pst))
data_k25=data[data.Popid=='K25']
print('K25 Data range:', min(data_k25.datetime_pst), " ~ ",max(data_k25.datetime_pst))
data_k33=data[data.Popid=='K33']
print('K33 Data range:', min(data_k33.datetime_pst), " ~ ",max(data_k33.datetime_pst))
data_l84=data[data.Popid=='L84']
print('L84 Data range:', min(data_l84.datetime_pst), " ~ ",max(data_l84.datetime_pst))
data_l87=data[data.Popid=='L87']
print('L87 Data range:', min(data_l87.datetime_pst), " ~ ",max(data_l87.datetime_pst))
data_l88=data[data.Popid=='L88']
print('L88 Data range:', min(data_l88.datetime_pst), " ~ ",max(data_l88.datetime_pst))
data_l95=data[data.Popid=='L95']
print('L95 Data range:', min(data_l95.datetime_pst), " ~ ",max(data_l95.datetime_pst))

J26 Data range: 2012-02-20 17:50:02-08:00  ~  2012-02-23 10:13:58-08:00
J27 Data range: 2014-12-28 14:56:26-08:00  ~  2015-02-15 10:52:57-08:00
K25 Data range: 2012-12-29 13:40:16-08:00  ~  2013-04-03 21:38:20-07:00
K33 Data range: 2015-12-31 16:17:11-08:00  ~  2016-02-17 12:27:05-08:00
L84 Data range: 2015-02-17 13:38:20-08:00  ~  2015-05-21 16:56:33-07:00
L87 Data range: 2013-12-26 13:29:49-08:00  ~  2014-01-26 06:38:42-08:00
L88 Data range: 2013-03-02 12:10:19-08:00  ~  2013-03-10 07:13:51-07:00
L95 Data range: 2016-02-23 14:41:42-08:00  ~  2016-02-26 06:26:53-08:00


Add a numerical day with 0 being the day of the earliest record for each orca

In [45]:
def orcaday(dat):
    day0=min(dat['date'])
    dat['day_move']=dat['date'].apply(lambda x: x-day0)

Use alternative longitude and latitude values when the primary values are out of bound

In [56]:
def geogen(dat):
    import numpy as np
    dat['lon']=np.where(dat['lon_p']>-120, dat['lon_a'],dat['lon_p'])
    dat['lat']=np.where(dat['lon_p']>-120, dat['lat_a'],dat['lat_p'])

Visualize geo location by pod

In [86]:
def orcamove(dat):
    orcaday(dat)
    geogen(dat)
    lon_avg=dat['lon'].mean()
    lat_avg=dat['lat'].mean()
    mapbox_access_token = open(pjoin(base_path,"mapbox_token.txt")).read()
    fig = go.Figure()
    fig.add_trace(go.Scattermapbox(
            lat=dat['lat'],
            lon=dat['lon'],
            mode='markers+lines',
            marker=go.scattermapbox.Marker(
                color=dat.day_move,
                size=7,
                colorscale='Blues',
                opacity=0.75,
            ),        
            text=dat['datetime_pst'],
            hoverinfo='text'
        ))
    fig.update_layout(
        autosize=True,
        showlegend=False,
        margin=dict(l=0, r=0, t=0, b=0),
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=dict(
                lat=lat_avg,
                lon=lon_avg
            ),
            zoom=4,
            style='dark'
        ),
    )    
    fig.show()

J26

In [76]:
orcamove(data_j26)

J27

In [68]:
orcamove(data_j27)

K25

In [78]:
orcamove(data_k25)

K33

In [79]:
orcamove(data_k33)

L84

In [87]:
orcamove(data_l84)

L87

In [72]:
orcamove(data_l87)

L88

In [84]:
orcamove(data_l88)

L95

In [83]:
orcamove(data_l95)